# Install dependencies


In [ ]:
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-release-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

--2025-12-18 13:49:12--  https://fem-on-colab.github.io/releases/fenicsx-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4339 (4.2K) [application/x-sh]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[===================>]   4.24K  --.-KB/s    in 0s      

2025-12-18 13:49:12 (40.3 MB/s) - ‘/tmp/fenicsx-install.sh’ saved [4339/4339]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICSX_INSTALLED=/usr/local/share/fem-on-colab/fenicsx.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenicsx.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1f62c6f6/release

In [ ]:
try:
  import polyhedral_net_splines as pns
except ImportError:
  from google.colab import files
  uploaded = files.upload()
  !unzip polyhedral_splines.zip -d polyhedral_splines
  !pip install cmake
  !rm -rf polyhedral_splines.zip
  %cd polyhedral_splines/polyhedral_splines
  !pip install .
  %cd ../..
  import polyhedral_net_splines as pns



Saving polyhedral_splines.zip to polyhedral_splines.zip
Archive:  polyhedral_splines.zip
   creating: polyhedral_splines/polyhedral_splines/
   creating: polyhedral_splines/polyhedral_splines/.git/
  inflating: polyhedral_splines/polyhedral_splines/.git/config  
  inflating: polyhedral_splines/polyhedral_splines/.git/description  
  inflating: polyhedral_splines/polyhedral_splines/.git/HEAD  
   creating: polyhedral_splines/polyhedral_splines/.git/hooks/
  inflating: polyhedral_splines/polyhedral_splines/.git/hooks/applypatch-msg.sample  
  inflating: polyhedral_splines/polyhedral_splines/.git/hooks/commit-msg.sample  
  inflating: polyhedral_splines/polyhedral_splines/.git/hooks/fsmonitor-watchman.sample  
  inflating: polyhedral_splines/polyhedral_splines/.git/hooks/post-update.sample  
  inflating: polyhedral_splines/polyhedral_splines/.git/hooks/pre-applypatch.sample  
  inflating: polyhedral_splines/polyhedral_splines/.git/hooks/pre-commit.sample  
  inflating: polyhedral_splines/

In [ ]:
import dolfinx
import basix
import ufl
from mpi4py import MPI
from dolfinx.fem.petsc import LinearProblem
import numpy
import numpy as np
import pyvista as pv
import inspect

In [ ]:
from mpi4py import MPI
from petsc4py import PETSc

import os
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import tqdm.autonotebook

from pyvista import examples
import imageio


import basix
from basix import CellType, ElementFamily, LagrangeVariant, SobolevSpace
from basix.ufl import element

import ufl
from ufl import (FacetNormal, Identity, Measure, TestFunction, TrialFunction,
                 as_vector, div, dot, ds, dx, inner, lhs, grad,
                 nabla_grad, rhs, sym, system, SpatialCoordinate)

from dolfinx import mesh, io, plot, fem
from dolfinx.mesh import (create_unit_square, CellType as DFXCellType,
                          create_unit_interval, create_mesh,
                          meshtags_from_entities, create_unit_cube, create_box)
from dolfinx.cpp.mesh import to_type, cell_entity_type
from dolfinx.cpp.io import perm_gmsh

from dolfinx.fem import (Constant, Function, functionspace,
                         assemble_scalar, dirichletbc, form,
                         locate_dofs_geometrical, locate_dofs_topological,
                         set_bc)
from dolfinx.fem.petsc import (apply_lifting, assemble_matrix,
                               assemble_vector, create_vector,
                               create_matrix, set_bc)
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import (VTXWriter, XDMFFile, distribute_entity_data)
from dolfinx.io import gmsh as dfx_gmesh
from dolfinx.plot import vtk_mesh
from dolfinx.graph import adjacencylist
from dolfinx.geometry import (bb_tree, compute_collisions_points,
                              compute_colliding_cells)

# PNS FEM

In [ ]:
import typing
from petsc4py import PETSc
import dolfinx.fem.petsc
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, create_matrix, _create_form, create_vector
import itertools
import functools
from mpi4py import MPI
import ufl
import numpy as np
import polyhedral_net_splines as pns
import basix
from tqdm import tqdm

import time
from functools import wraps
from contextlib import contextmanager

from dolfinx.fem import extract_function_spaces

# Store total times
total_times = {}

# Thread-local (or global) exclusion time store
_excluded_time = {}

@contextmanager
def exclude_time():
    thread_id = 'default'  # In real multi-threaded code, use threading.get_ident()
    start = time.perf_counter()
    yield
    end = time.perf_counter()
    _excluded_time[thread_id] = _excluded_time.get(thread_id, 0) + (end - start)

def track_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        thread_id = 'default'
        _excluded_time[thread_id] = 0  # Reset before each call
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        elapsed = end - start - _excluded_time[thread_id]
        total_times[func.__name__] = total_times.get(func.__name__, 0) + elapsed
        return result
    return wrapper

cubicLagrangeDofOrder = [0, 6, 7, 2, 4, 12, 14, 10, 5, 13, 15, 11, 1, 8, 9, 3]
cubicLagrangeDofOrderInv = [0, 12, 3, 15, 4, 8, 1, 2, 13, 14, 7, 11, 5, 9, 6, 10]

def assemble_prolongation_matrix(pnsObject) -> PETSc.Mat:
    """Construct Prolongation matrix of size (numBBcoeffs, numPnsCP)

    Args:
        pnsObject: PNS object

    Returns:
        Prolongation matrix
    """
    comm = pnsObject['comm']
    numBB = pnsObject['numPatches'] * 16
    numPns = pnsObject['numVerts']
    numElements = pnsObject['numElements']

    K = PETSc.Mat().createAIJ(size=(numBB*numElements, numPns*numElements), comm=comm)
    K.setUp()
    currCp = 0
    for pb in pnsObject['augmentedLagrangePatchBuilders']:
        neighborhood = pb.neighbor_verts
        mask = pb.mask
        for maskIdx in range(len(mask)):
            rowIdx = (maskIdx//16)*16 + cubicLagrangeDofOrderInv[maskIdx % 16]
            cpRow = mask[rowIdx]
            u, inv = np.unique(neighborhood, return_inverse=True)
            coeffs = dict(zip(u, np.bincount(inv, weights=cpRow)))
            for neighbor in coeffs:
                for e in range(numElements):
                    writeRow = e*16 + currCp
                    writeCol = e + numElements * neighbor
                    K[writeRow, writeCol] = coeffs[neighbor]
            if maskIdx % 16 == 15:
                currCp += (numElements-1)*16
            currCp += 1
    K.assemble()
    return K


def fix_zero_rows(mat: PETSc.Mat):
    """For any all-zero row in PETSc Mat, set the diagonal to 1."""
    nrows, _ = mat.getSize()
    zero_rows = []
    for i in range(nrows):
        # Get row entries
        cols, vals = mat.getRow(i)
        if cols.size == 0 or all(abs(v) < 1e-14 for v in vals):
            zero_rows.append(i)
    mat.zeroRowsColumns(zero_rows, diag = 1)
    mat.assemble()
    return mat

@track_time
def assemble_matrix_pns(pnsObject, a, bc = None, lagrange_bcs=[], result = None, A = None, AK = None, KTAK = None):
    """Assemble bilinear form into a matrix for PNS.
    Args:
        pnsObject: PNS object
        a: Bilinear UFL form or a sequence of sequence of bilinear
            forms, the left hand side of the variational problem.
        bc: Dirichlet boundary condition with PNS DOF.
        lagrange_bcs: Lagrange basis Dirichlet boundary conditions.
        result: Optional Matrix to assemble the bilinear form into.
        A: Bilinear form matrix in Lagrange basis
        AK: Bilinear form matrix in Lagrange basis
        KTAK: Temprory as setting to result directly without copy not working.
    Returns:
        Bilinear form matrix for PNS.
        Bilinear form matrix in Lagrange basis(no BC)
    """
    comm = pnsObject['comm']

    with exclude_time():
        if A is None:
            A = create_matrix(a)
        # A.assemble()
        A.zeroEntries()
        A = dolfinx.fem.petsc.assemble_matrix(A, a, bcs=lagrange_bcs)
        A.assemble()
    K = pnsObject['prolongation_matrix']
    AK = A.matMult(K, result=AK)
    KTAK = K.transposeMatMult(AK, result = KTAK) # for some reason cannot set directly to result when result is used with non-linear solver.
    if bc is not None:
        bc.setMatrix(KTAK)
    KTAK.assemble()
    KTAK = fix_zero_rows(KTAK)
    if result is not None:
        result.assemble()
        KTAK.copy(result)
    return KTAK, A

@track_time
def assemble_vector_pns(pnsObject, L, A = None, bc = None, lagrange_bcs=[], bilinear_form=None, result = None, u_func_for_lifting=None, alpha=1.0, b = None): # Renamed x0 to u_func_for_lifting
    """Assemble linear form into a vector for PNS.
    Args:
        pnsObject: PNS object
        L: Linear UFL form or a sequence of linear forms, the right
            hand side of the variational problem.
        A: Bilinear form matrix in Lagrange basis
        bc: Dirichlet boundary condition with PNS DOF.
        lagrange_bcs: Lagrange basis Dirichlet boundary conditions.
        bilinear_form: Bilinear form in Lagrange basis. Required if applying lagrange bcs.
        result: Optional Vector to assemble the linear form into.
        u_func_for_lifting: Current solution as dolfinx.fem.Function. Used for lifting BCs.
        alpha: For BC
        b: Optional vector to store the temporary vector for Lagrange basis.
    Returns:
        Linear form vector for PNS.
    """
    comm = pnsObject['comm']
    if result is None:
        result = PETSc.Vec().createSeq(pnsObject['prolongation_matrix'].getSize()[1], comm=comm)
    K = pnsObject['prolongation_matrix']
    with exclude_time():
        if b is None:
            #Argument from L -> extract_function_spaces(L)
            b = create_vector(extract_function_spaces(L))
        with b.localForm() as bl:
            bl.set(0)
        dolfinx.fem.petsc.assemble_vector(b, L)
        if len(lagrange_bcs) > 0:
            dolfinx.fem.petsc.apply_lifting(b, [bilinear_form], [lagrange_bcs], x0=[u_func_for_lifting] if u_func_for_lifting is not None else [], alpha=alpha)
            dolfinx.fem.petsc.set_bc(b, lagrange_bcs, x0=u_func_for_lifting, alpha=alpha)
            #x0_arg_for_lifting = [u_func_for_lifting.x.petsc_vec] if u_func_for_lifting is not None else []
            #dolfinx.fem.petsc.apply_lifting(b, [bilinear_form], [lagrange_bcs], x0=x0_arg_for_lifting, alpha=alpha)
            #dolfinx.fem.petsc.set_bc(b, lagrange_bcs, x0=u_func_for_lifting.x.petsc_vec, alpha=alpha)

    lifted_b = b # if no pns BC

    # TODO: account for x0 and alpha
    if bc is not None:
        AK = A.matMult(K)
        x_g = bc.getXg()
        lifted_b = PETSc.Vec().createSeq(K.getSize()[0], comm=comm)
        AK.mult(x_g, lifted_b)
        lifted_b.aypx(-1, b)
    K.multTranspose(lifted_b, result)
    if bc is not None:
        bc.setVector(result)
    result.assemble()
    return result, b

def pnsAssign(source_pns: PETSc.Vec, target_lagrange: dolfinx.fem.Function, pnsObject):
    """Assign the value from source vector in Lagrange basis to target vector in PNS basis.
    Args:
        source_pns: Source vector in PNS basis.
        target_lagrange: Target function in Lagrange basis.
        pnsObject: PNS object
    """
    K = pnsObject['prolongation_matrix']
    K.mult(source_pns, target_lagrange.x.petsc_vec)

def pnsLagrangeAssign(source_lagrange: dolfinx.fem.Function, target_pns: PETSc.Vec, pnsObject):
    """Assign the value from source vector in PNS basis to target vector in Lagrange basis.
    Args:
        source_lagrange: Source function in PNS basis.
        target_pns: Target vector in Lagrange basis.
        pnsObject: PNS object
    """
    K = pnsObject['prolongation_matrix']
    K.multTranspose(source_lagrange.x.petsc_vec, target_pns)

class PnsLinearProblem:
    def __init__(
        self,
        a: ufl.Form,
        L: ufl.Form,
        pnsObject,
        bc: typing.Optional[dolfinx.fem.DirichletBC] = None,
        lagrange_bcs = [],
        comm = MPI.COMM_WORLD,
        petsc_options: typing.Optional[dict] = None,
        form_compiler_options: typing.Optional[dict] = None,
        jit_options: typing.Optional[dict] = None,
    ):
        """Initialize solver for a linear variational problem.

        Args:
            a: Bilinear UFL form or a sequence of sequence of bilinear
                forms, the left hand side of the variational problem.
            L: Linear UFL form or a sequence of linear forms, the right
                hand side of the variational problem.
            pnsObject: PNS object
            bc(PnsDirichletBC): Dirichlet boundary condition with PNS DOF.
            lagrange_bcs: Lagrange basis Dirichlet boundary conditions.
            petsc_options: Options that are passed to the linear
                algebra backend PETSc. For available choices for the
                'petsc_options' kwarg, see the `PETSc documentation
                <https://petsc4py.readthedocs.io/en/stable/manual/ksp/>`_.
            form_compiler_options: Options used in FFCx compilation of
                all forms. Run ``ffcx --help`` at the commandline to see
                all available options.
            jit_options: Options used in CFFI JIT compilation of C
                code generated by FFCx. See `python/dolfinx/jit.py` for
                all available options.

                available options. Takes priority over all other
                option values.

        Example::

            problem = LinearProblem(a, L, bc, petsc_options={
                "ksp_type": "preonly",
                "pc_type": "lu",
                "pc_factor_mat_solver_type": "mumps"
            })
        """
        # Maybe u needs to be in pns for.
        # Create K. Maybe k can be represended as a form then create_matrix function can be used
        self.pnsObject = pnsObject
        self._a = dolfinx.fem.form(a)
        self._L = dolfinx.fem.form(L)

        self.bc = bc
        self.lagrange_bcs = lagrange_bcs

        comm = pnsObject['comm']

        self._solver = PETSc.KSP().create(comm)
        prefix = f"dolfinx_solve_{id(self)}"
        self._solver.setOptionsPrefix(prefix)

        opts = PETSc.Options()
        opts.prefixPush(prefix)
        if petsc_options:
            for key, val in petsc_options.items():
                opts[key] = val
        opts.prefixPop()
        self._solver.setFromOptions()

    def solve(self) -> PETSc.Vec:
        A_reduced, A = assemble_matrix_pns(self.pnsObject, self._a, self.bc, self.lagrange_bcs)
        b_reduced, _ = assemble_vector_pns(self.pnsObject, self._L, A, self.bc, self.lagrange_bcs, self._a, u_func_for_lifting=None) # No solution function available here for linear problem
        self._solver.setOperators(A_reduced)
        x_reduced = b_reduced.duplicate()
        self._solver.solve(b_reduced, x_reduced)
        return x_reduced

    def __del__(self):
        return

class PnsNewtonSolver(dolfinx.cpp.nls.petsc.NewtonSolver):
    def __init__(self, comm: MPI.Intracomm, problem):
        """A Newton solver for non-linear problems."""
        super().__init__(comm)
        self.problem = problem
        # Create matrix and vector to be used for assembly
        # of the non-linear problem
        pnsObject = problem.pnsObject
        numPnsDof = pnsObject['prolongation_matrix'].getSize()[1]
        self._A = PETSc.Mat().createAIJ(size=(numPnsDof, numPnsDof), comm=comm)
        self.setJ(problem.J, self._A)
        self._b = PETSc.Vec().createSeq(numPnsDof, comm=comm)
        self.setF(problem.F, self._b)
        self._x = PETSc.Vec().createSeq(numPnsDof, comm=comm)
        pnsObject['prolongation_matrix'].multTranspose(problem.u.x.petsc_vec, self._x)
        self.set_form(problem.form)

    def __del__(self):
        self._A.destroy()
        self._b.destroy()

    def solve(self, u: dolfinx.fem.Function):
        """Solve non-linear problem into function u. Returns the number
        of iterations and if the solver converged."""
        n, converged = super().solve(self._x)
        self.problem.pnsObject['prolongation_matrix'].mult(self._x, u.x.petsc_vec)
        u.x.scatter_forward()
        return n, converged

    @property
    def A(self) -> PETSc.Mat:  # type: ignore
        """Jacobian matrix"""
        return self._A

    @property
    def b(self) -> PETSc.Vec:  # type: ignore
        """Residual vector"""
        return self._b

    def setP(self, P: dolfinx.fem.Function, Pmat: PETSc.Mat):  # type: ignore
        """
        Set the function for computing the preconditioner matrix

        Args:
            P: Function to compute the preconditioner matrix
            Pmat: Matrix to assemble the preconditioner into

        """
        super().setP(P, Pmat)




class PnsNonlinearProblem:
    """Nonlinear problem class for solving the non-linear problems.

    Solves problems of the form :math:`F(u, v) = 0 \ \forall v \in V` using
    PETSc as the linear algebra backend.
    """

    def __init__(
        self,
        F: ufl.form.Form,
        u: dolfinx.fem.Function,
        pnsObject,
        bcs: list[dolfinx.fem.DirichletBC] = [],
        J: ufl.form.Form = None,
        form_compiler_options: typing.Optional[dict] = None,
        jit_options: typing.Optional[dict] = None,
    ):
        """Initialize solver for solving a non-linear problem using Newton's method`.

        Args:
            F: The PDE residual F(u, v)
            u: The unknown
            bcs: List of Dirichlet boundary conditions
            J: UFL representation of the Jacobian (Optional)
            form_compiler_options: Options used in FFCx
                compilation of this form. Run ``ffcx --help`` at the
                command line to see all available options.
            jit_options: Options used in CFFI JIT compilation of C
                code generated by FFCx. See ``python/dolfinx/jit.py``
                for all available options. Takes priority over all
                other option values.

        Example::

            problem = LinearProblem(F, u, [bc0, bc1])
        """
        self.u = u
        self._L = _create_form(
            F, form_compiler_options=form_compiler_options, jit_options=jit_options
        )

        # Create the Jacobian matrix, dF/du
        if J is None:
            V = extract_function_spaces(u) #u.function_space -> extract_function_spaces(u)
            du = ufl.TrialFunction(V)
            J = ufl.derivative(F, u, du)

        self._a = _create_form(
            J, form_compiler_options=form_compiler_options, jit_options=jit_options
        )
        self.bcs = bcs
        self.pnsObject = pnsObject
        self.bTemp = create_vector(self._L)
        self.Atemp = create_matrix(self._a)
        self.AKtemp = PETSc.Mat()
        self.KTAKtemp = PETSc.Mat()

    @property
    def L(self) -> ufl.form.Form:
        """Compiled linear form (the residual form)"""
        return self._L

    @property
    def a(self) -> ufl.form.Form:
        """Compiled bilinear form (the Jacobian form)"""
        return self._a

    def form(self, x: PETSc.Vec) -> None:
        """This function is called before the residual or Jacobian is
        computed. This is usually used to update ghost values.

        Args:
           x: The vector containing the latest solution
        """
        # x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        self.pnsObject['prolongation_matrix'].mult(x, self.u.x.petsc_vec)

    def F(self, x: PETSc.Vec, b: PETSc.Vec) -> None:
        """Assemble the residual F into the vector b.

        Args:
            x: The vector containing the latest solution
            b: Vector to assemble the residual into
        """
        # # Reset the residual vector
        # with b.localForm() as b_local:
        #     b_local.set(0.0)
        # assemble_vector(b, self._L)

        # # Apply boundary condition
        # apply_lifting(b, [self._a], bcs=[self.bcs], x0=[x], alpha=-1.0)
        # b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        # set_bc(b, self.bcs, x, -1.0)

        assemble_vector_pns(self.pnsObject, self._L, lagrange_bcs=self.bcs, bilinear_form=self._a, result=b, u_func_for_lifting=self.u, alpha=-1.0, b=self.bTemp)
        # print(b.array)
        # print("----------------\n\n\n-------------------")
        # print("Residual norm:", b.norm())

    def J(self, x: PETSc.Vec, A: PETSc.Mat) -> None:
        """Assemble the Jacobian matrix.

        Args:
            x: The vector containing the latest solution
        """
        # A.zeroEntries()
        # assemble_matrix(A, self._a, self.bcs)
        # A.assemble()
        assemble_matrix_pns(self.pnsObject, self._a, lagrange_bcs=self.bcs, result=A, A=self.Atemp, AK=self.AKtemp, KTAK=self.KTAKtemp)

class PnsDirichletBC:
    def __init__(self, pnsObject, value: np.ndarray, dof: np.ndarray):
        self.value = value
        self.dof = dof
        self.pnsObject = pnsObject

    def setVector(self, x: PETSc.Vec):
        x.setValues(self.dof, self.value)
        x.assemble()

    def setMatrix(self, mat: PETSc.Mat):
        mat.zeroRowsColumns(self.dof, diag = 1)
        mat.assemble()
        # mat.zeroRows(self.dof, diag=1)

    def getXg(self):
        x_g = PETSc.Vec().createSeq(self.pnsObject['prolongation_matrix'].getSize()[1], comm=self.pnsObject['comm'])
        x_g.setValues(self.dof, self.value)
        x_g.assemble()
        return x_g

def loadPns(file_name: str, numElements = 1, set_boundary_gradient=False, comm = MPI.COMM_WORLD):
    control_mesh = pns.Pns_control_mesh.from_file(file_name)
    if set_boundary_gradient:
        augmented_control_mesh = pns.set_boundary_gradient(control_mesh)
    else:
        augmented_control_mesh = control_mesh
    pnsLagrangePatchBuilders = pns.get_patch_builders(control_mesh)
    pnsPatchBuilders = pns.get_patch_builders(control_mesh)
    pnsAugmentedLagrangePatchBuilders = pns.get_patch_builders(augmented_control_mesh)
    pnsAugmentedPatchBuilders = pns.get_patch_builders(augmented_control_mesh)
    if len(pnsLagrangePatchBuilders) == 0:
        raise Exception("No patches found")
    flatPatches = []
    numPatches = 0
    for pb in pnsLagrangePatchBuilders:
        pb.toLagrange()
    for pb in pnsAugmentedLagrangePatchBuilders:
        pb.toLagrange()
    for pb in pnsPatchBuilders:
        for patch in pb.build_patches(control_mesh):
            flatPatches.append(patch)
            numPatches += 1
        pb.degRaise()
    for pb in pnsAugmentedPatchBuilders:
        pb.degRaise()
    numVerts = len(control_mesh.get_vertices())
    pnsObject = {'lagrangePatchBuilders': pnsLagrangePatchBuilders, 'patchBuilders': pnsPatchBuilders, 'augmentedLagrangePatchBuilders': pnsAugmentedLagrangePatchBuilders, 'augmentedPatchBuilders': pnsAugmentedPatchBuilders, 'numPatches': numPatches, 'numVerts': numVerts, 'control_mesh': control_mesh, "flat_patches": flatPatches, "numElements": numElements, "comm": comm}
    K = assemble_prolongation_matrix(pnsObject)
    pnsObject["prolongation_matrix"] = K
    return pnsObject

def createPnsDolfinxMesh(pnsObject) -> dolfinx.mesh.Mesh:
    x = []
    cells = []
    idx = 0
    temp = 0
    for pb in pnsObject['lagrangePatchBuilders']:
        lagrangePatches = pb.build_patches(pnsObject['control_mesh'])
        for patch in lagrangePatches:
            temp += 1
            lagrangeCoeffs = patch.bb_coefs
            currX = [0]*16
            cpIdx = 0
            for row in lagrangeCoeffs:
                for col in row:
                    currX[cubicLagrangeDofOrder[cpIdx]] = [col[0], col[1], col[2]]
                    cpIdx += 1
            x.extend(currX)
            cells.append(list(range(idx, idx+16)))
            idx += 16
    domain = ufl.Mesh(
        basix.ufl.element(
            "Lagrange",
            "quadrilateral",
            3,
            lagrange_variant=basix.LagrangeVariant.equispaced,
            shape=(3,),
        )
    )

    mesh = dolfinx.mesh.create_mesh(pnsObject["comm"], cells, domain, x)
    return mesh

def pnsFunctionSpace(mesh, pnsObject) -> dolfinx.fem.FunctionSpace:
    el = basix.ufl.element(
            "Lagrange",
            "quadrilateral",
            3,
            lagrange_variant=basix.LagrangeVariant.equispaced,
        )
    if pnsObject['numElements'] > 1:
        el = basix.ufl.mixed_element([el] * pnsObject['numElements'])
    V = dolfinx.fem.functionspace(mesh, el)
    return V



# testProlongationMatrix(pnsObject)

# """## Visualize Pns"""


import numpy as np
import json
from math import comb
from functools import cache

class PnsVisualizer:
    def __init__(self, pnsObject, dt=0):
        self.pnsObject = pnsObject
        self.dt = dt
        self.funcList = []

    def addFunc(self, valDofs):
        self.funcList.append(valDofs.copy())

    def visualizePns(self, filename, val_range=None):
        """
        Generates a file that can be visualized by https://cise.ufl.edu/~p.gupta/pns-fea-visualizer/
        """

        allPatchCps = np.array([
            patch.bb_coefs
            for pb in self.pnsObject['patchBuilders']
            for patch in pb.build_patches(self.pnsObject['control_mesh'])
        ])  # shape (P,4,4,3)

        num_patches = allPatchCps.shape[0]
        num_timesteps = len(self.funcList)
        # build val‐function control‐point arrays
        all_valPatchCps = np.array([
            patch.bb_coefs
            for valDofs in self.funcList
            for pb in self.pnsObject['augmentedPatchBuilders']
            for patch in pb.build_patches(
                pns.Pns_control_mesh.from_data(
                    [(v,0,0) for v in valDofs],
                    []
                )
            )
        ])  # shape (num_timesteps*P,4,4,3)
        all_valPatchCps = all_valPatchCps[:, :, :, 0]  # take x‐coord as scalar value
        all_valPatchCps = all_valPatchCps.reshape(num_timesteps, num_patches, 4, 4)  # shape (num_timesteps,P,4,4)
        if val_range is None:
            min_val = np.min(all_valPatchCps)
            max_val = np.max(all_valPatchCps)
        else:
            min_val, max_val = val_range
        out = [str(num_patches), str(num_timesteps), str(min_val), str(max_val)]

        for p in range(num_patches):
            out.append("3 3")  # u‐degree, v‐degree
            out.append(" ".join(map(str, allPatchCps[p].flatten().tolist())))
            out.append(" ".join(map(str, all_valPatchCps[:, p].flatten().tolist())))

        out_str = "\n".join(out)

        with open(filename, "w") as f:
            f.write(out_str)

<>:344: SyntaxWarning: invalid escape sequence '\ '
<>:344: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-3266781016.py:344: SyntaxWarning: invalid escape sequence '\ '
  Solves problems of the form :math:`F(u, v) = 0 \ \forall v \in V` using


In [ ]:
class PnsMatrixNonlinearSolver(PETSc.KSP):
    def __init__(self,
                 comm: MPI.Intracomm,
                 pnsObject,
                 lhs: ufl.form.Form, # UFL form for Jacobian (e.g., a1)
                 rhs: ufl.form.Form, # UFL form for residual L (e.g., L1)
                 F: ufl.form.Form, # UFL form for the full residual (e.g., F1)
                 u: dolfinx.fem.Function, # The dolfinx.fem.Function to be solved for (e.g., u_, p_)
                 J: ufl.form.Form = None, # UFL form for Jacobian (if explicitly provided)
                 bcs: list[dolfinx.fem.DirichletBC] = [],
                 form_compiler_options: typing.Optional[dict] = None,
                 jit_options: typing.Optional[dict] = None):
        """A Newton solver for non-linear problems."""
        """Solves Nonlinear Ax = b problems

        solver2 = PETSc.KSP().create(mesh.comm)
        solver2.setOperators(A2)
        solver2.setType(PETSc.KSP.Type.BCGS)
        pc2 = solver2.getPC()
        pc2.setType(PETSc.PC.Type.HYPRE)
        pc2.setHYPREType("boomeramg")
        """
        super().create(comm)
        self.pnsObject = pnsObject
        self.u = u
        self.bcs = bcs
        self._F = F
        if J is None:
            V = self.u.function_space
            du = ufl.TrialFunction(V)
            self._J = ufl.derivative(self._F, self.u, du) # UFL form for F1
        else:
            self._J = J


        # Create matrix and vector to be used for assembly
        # of the non-linear problem
        pnsObject = pnsObject
        numPnsDof = pnsObject['prolongation_matrix'].getSize()[1]
        self._A = PETSc.Mat().createAIJ(size=(numPnsDof, numPnsDof), comm=comm)
        self.setJ(self._J, self._A)
        self._b = PETSc.Vec().createSeq(numPnsDof, comm=comm)
        self.setF(self._F, self._b)
        self._x = PETSc.Vec().createSeq(numPnsDof, comm=comm)
        pnsObject['prolongation_matrix'].multTranspose(self.u.x.petsc_vec, self._x)
        self.set_form(self._F)

    def __del__(self):
        self._A.destroy()
        self._b.destroy()

    def solve(self, u: dolfinx.fem.Function):
        """Solve non-linear problem into function u. Returns the number
        of iterations and if the solver converged."""
        n, converged = super().solve(self._x)
        self.problem.pnsObject['prolongation_matrix'].mult(self._x, u.x.petsc_vec)
        u.x.scatter_forward()
        return n, converged

    @property
    def A(self) -> PETSc.Mat:  # type: ignore
        """Jacobian matrix"""
        return self._A

    @property
    def b(self) -> PETSc.Vec:  # type: ignore
        """Residual vector"""
        return self._b

    def setP(self, P: dolfinx.fem.Function, Pmat: PETSc.Mat):  # type: ignore
        """
        Set the function for computing the preconditioner matrix

        Args:
            P: Function to compute the preconditioner matrix
            Pmat: Matrix to assemble the preconditioner into

        """
        super().setP(P, Pmat)

#Navier Stokes PnS w/ Fenicsx


In [ ]:

from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista as pv

from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical, locate_dofs_topological
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter
from dolfinx.mesh import create_unit_square
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction,TrialFunctions, TestFunctions,
                 div, dot, ds, dx, inner, lhs, nabla_grad, rhs, sym)
from dolfinx.mesh import locate_entities_boundary, locate_entities


In [ ]:
# Copyright (C) 2023 Jørgen S. Dokken and Halvor Herlyng
#
# SPDX-License-Identifier:    MIT
from normals_and_tangents import *

In [ ]:
"""
Adapted from
Jørgen S. Dokken, Test Problem 1: Channel flow(Poiseuille), https://jsdokken.com/dolfinx-tutorial/chapter2/ns_code1.html
"""
!ls
pnsObject = loadPns("airplane.obj", numElements=3)
pnsObject2 = loadPns("airplane.obj")
mesh = createPnsDolfinxMesh(pnsObject)
tdim = mesh.topology.dim
fdim = mesh.topology.dim-1
mesh.topology.create_connectivity(fdim, mesh.topology.dim)

airplane.obj  normals_and_tangents.py  Pressure     sample_data
Flow	      polyhedral_splines       __pycache__


In [ ]:
V = pnsFunctionSpace(mesh, pnsObject)
Q = pnsFunctionSpace(mesh, pnsObject2)

In [ ]:
"""
Constants
"""
t = 5 #T_0
T = 10 #T_1
num_steps = 50
dt = dolfinx.fem.Constant(mesh, (T-t) / num_steps)
pi = 3.141593653589
err = 1e-8 #Error

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

In [ ]:
coords = mesh.geometry.x     # shape (num_vertices, gdim)
bmin   = coords.min(axis=0) # array([min_x, min_y, min_z])
bmax   = coords.max(axis=0)
print(bmin, bmax)

[-9.60034752 -3.96267653 -0.95872593] [1.57978475 3.96047568 2.36406612]


In [ ]:
"""
Computes Subspace information
"""
V0 = V.sub(0)
V1 = V.sub(1)
V2 = V.sub(2)
V0_c, map0 = V0.collapse()
V1_c, map1 = V1.collapse()
V2_c, map2 = V2.collapse()

In [ ]:
"""
Find Facet Normal vectors, Doesnt Find all due to Less Vertices on Pns Mesh :(
"""
facets_normal = locate_entities(mesh, fdim, lambda x: np.isclose(x[0], x[0]))
ones = np.ones(len(facets_normal), dtype=np.int32)
facet_tags = dolfinx.mesh.meshtags(mesh, tdim, facets_normal, ones)
tangent, interior = False, False
n0, n1, n2 = facet_vector_approximation(V, facet_tags, 1, tangent, interior)
"""
Constructs Parent Function
"""
nP = dolfinx.fem.Function(V)
nP.x.array[V.sub(0).dofmap.list.flatten()] = n0.x.array
nP.x.array[V.sub(1).dofmap.list.flatten()] = n1.x.array
nP.x.array[V.sub(2).dofmap.list.flatten()] = n2.x.array

In [ ]:
"""
Finds Coordinates of Collapes Subspaces, Mapes them back to Coordinate on original Mesh
"""
coords0, coords1, coords2 = V0_c.tabulate_dof_coordinates(), V1_c.tabulate_dof_coordinates(), V2_c.tabulate_dof_coordinates()
x, y, z = nP.x.array[map0], nP.x.array[map1], nP.x.array[map2]
values = [[], [], []]
vector_values = [[], [], []]
for i in range(len(coords0)):
  if np.isclose(x[i], -1.0, rtol = 0.05) and np.isclose(coords0[i][0], bmin[0], rtol=0.05):
    values[0].append(coords0[i][0])
    values[1].append(coords1[i][1])
    values[2].append(coords2[i][2])
    vector_values[0].append(x[i])
    vector_values[1].append(y[i])
    vector_values[2].append(z[i])
while len(values[0]) < 4188:
  values[0].append(-11)
  values[1].append(-11)
  values[2].append(-11)



In [ ]:
def closen0(x):
  return np.isclose(x[0], values[0], rtol=0.16)
def closen1(x):
  return np.isclose(x[1], values[1], rtol=0.2)
def closen2(x):
  return np.isclose(x[2], values[2], rtol=0.2)

In [ ]:
"""
Flow Boundary Conditions
"""
facets_1 = locate_entities(mesh, fdim, closen0)
dofs1 = locate_dofs_topological(V0, fdim, facets_1)
u0 = dolfinx.fem.Function(V0_c)
u0.interpolate(lambda x: np.sin(x[0]) + np.cos(x[0]) + 10)
bc1 = dirichletbc(u0, dofs1)

facets_2 = locate_entities(mesh, fdim, closen1)
dofs2 = locate_dofs_topological(V1, fdim, facets_2)
u1 = dolfinx.fem.Function(V1_c)
u1.interpolate(lambda x: x[0] * np.cos(x[0]))
bc2 = dirichletbc(u1, dofs2)

facets_3 = locate_entities(mesh, fdim, closen2)
dofs3 = locate_dofs_topological(V2, fdim, facets_3)
u2 = dolfinx.fem.Function(V2_c)
u2.interpolate(lambda x: x[0] * np.sin(x[0]))
bc3 = dirichletbc(u2, dofs3)
print(len(facets_1), len(facets_2), len(facets_3))
bcu = [bc1, bc2, bc3]

56 0 0


In [ ]:
"""
Pressure Boundary Conditions
"""
scalar_random = np.random.uniform(0, 1, size = u0.x.array.size)
def pressure(x):
  return np.isclose(x[0], x[0])
facets_4 = locate_entities(mesh, fdim, pressure)
dofs4 = locate_dofs_topological(Q, fdim, facets_4)
v0 = dolfinx.fem.Function(Q)
v0.interpolate(lambda x: (x[0] * 0) + 1)
v0.x.array[:] *= scalar_random
bc4 = dirichletbc(v0, dofs4)
print(len(facets_4))
bcp = [bc4]

4188


In [ ]:
"""
From Channel Flow
"""
u_n = Function(V)
u_n.name = "u_n"
U = 0.5 * (u_n + u)
n = FacetNormal(mesh)
f = Constant(mesh, PETSc.ScalarType((1, 1, 1))) #Forcing Function
k = Constant(mesh, PETSc.ScalarType(dt))
mu = Constant(mesh, PETSc.ScalarType(1)) #Linear Elasticity
rho = Constant(mesh, PETSc.ScalarType(1)) #Density

In [ ]:
def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor
def sigma(u, p):
    return 2 * mu * epsilon(u) - p * Identity(len(u))


# Define the variational problem for the first step
p_n = Function(Q)
p_n.name = "p_n"
F1 = (rho * dot((u - u_n)/k, v) * dx
      + rho * dot(dot(u_n, nabla_grad(u_n)), v) * dx
      + inner(sigma(U, p_n), epsilon(v)) * dx
      + dot(p_n * n, v) * ds
      - dot(mu * nabla_grad(U) * n, v) * ds
      - dot(f, v) * dx
      )
a1 = form(lhs(F1))
L1 = form(rhs(F1))

# Define variational problem for step 2
u_ = Function(V)
a2 = form(dot(nabla_grad(p), nabla_grad(q)) * dx)
L2 = form(dot(nabla_grad(p_n), nabla_grad(q)) * dx - (rho / k) * div(u_) * q * dx)

# Define variational problem for step 3
p_ = Function(Q)
a3 = form(rho * dot(u, v) * dx)
L3 = form(rho * dot(u_, v) * dx - k * dot(nabla_grad(p_ - p_n), v) * dx)

In [ ]:
"""
PETSc Options for Solvers
"""
petsc_options_1 = {"setType":PETSc.KSP.Type.BCGS, "pc_type":PETSc.PC.Type.HYPRE, "pc_hypre_type":"boomeramg"}
petsc_options_2 = {"setType":PETSc.KSP.Type.BCGS, "pc_type":PETSc.PC.Type.HYPRE, "pc_hypre_type":"boomeramg"}
petsc_options_3 = {"setType":PETSc.KSP.Type.CG, "pc_type":PETSc.PC.Type.SOR}

In [ ]:
"""
Create solvers
"""
problem1 = PnsLinearProblem(a1, L1, pnsObject, lagrange_bcs=bcu, petsc_options=petsc_options_1)
problem2 = PnsLinearProblem(a2, L2, pnsObject2, lagrange_bcs=bcp, petsc_options=petsc_options_2)
problem3 = PnsLinearProblem(a3, L3, pnsObject, lagrange_bcs=bcu, petsc_options=petsc_options_3)

In [ ]:
"""
Creates Pns Visualization
"""
flow_vis = PnsVisualizer(pnsObject)
pressure_vis = PnsVisualizer(pnsObject2)

In [ ]:
for i in range(num_steps+1):
    # Step 1: Tentative veolcity step
    u_reduced_1 = problem1.solve()
    u_.x.scatter_forward()
    # Step 2: Pressure corrrection step
    p_reduced = problem2.solve()
    p_.x.scatter_forward()
    # Step 3: Velocity correction step
    u_reduced = problem3.solve()
    u_.x.scatter_forward()

    arr = np.array([u_reduced[i] for i in range(0, u_reduced.getSize(), 3)],
               dtype=PETSc.ScalarType)

    u_x_reduced = PETSc.Vec().createWithArray(arr, comm=PETSc.COMM_SELF)

    #Pressure Visualization
    flow_vis.addFunc(u_x_reduced.array.real)
    pressure_vis.addFunc(p_reduced.array.real)

    # Update variable with solution form this time step
    pnsLagrangeAssign(u_n, u_reduced, pnsObject)
    pnsLagrangeAssign(p_n, p_reduced, pnsObject2)

    p_err = np.linalg.norm(p_reduced.array.real)
    u_err = np.linalg.norm(u_reduced.array.real)
    u1_err = np.linalg.norm(u_reduced_1.array.real)

    #Interpolate the time dependent Boundary Conditions
    p_random = np.abs(np.random.randn(u0.x.array.size))
    u0.interpolate(lambda x: np.sin(x[0]) + np.cos(x[0]) + 10 * t)
    v0.interpolate(lambda x: (0 * x[0]) + 1)
    v0.x.array[:]*=p_random

    print(f"Flow @ {float(t):.3f}")
    # Update current time step
    t += float(dt.value)


In [ ]:
#Make Gif
flow_vis.visualizePns('Flow')
pressure_vis.visualizePns('Pressure')

#Navier Stokes Fenicsx


In [ ]:

from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista

from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter
from dolfinx.mesh import create_unit_square
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction,
                 div, dot, ds, dx, inner, lhs, nabla_grad, rhs, sym)


In [ ]:
"""
Adapted from
Jørgen S. Dokken, Test Problem 1: Channel flow(Poiseuille), https://jsdokken.com/dolfinx-tutorial/chapter2/ns_code1.html
"""
!ls
comm = MPI.COMM_WORLD
nx = 10
ny = 10
nz = 10
mesh = create_box(comm, [[0, 0, 0], [1, 1, 1]], [nx, ny, nz], DFXCellType.hexahedron)
finite_element_1 = basix.ufl.element(
    "Lagrange",
    "hexahedron",
    1,
    lagrange_variant=basix.LagrangeVariant.equispaced,
)
finite_element_3 = basix.ufl.element(
    "Lagrange",
    "hexahedron",
    1,
    shape = (mesh.geometry.dim,),
    lagrange_variant=basix.LagrangeVariant.equispaced,
)

In [ ]:
V = functionspace(mesh, finite_element_3)
Q = functionspace(mesh, finite_element_1)

In [ ]:
t = 0 #T_0
T = 1 #T_1
num_steps = 100
dt = dolfinx.fem.Constant(mesh, T / num_steps)
pi = 3.141593653589
err = 1e-8 #Error

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

In [ ]:
coords = mesh.geometry.x     # shape (num_vertices, gdim)
bmin   = coords.min(axis=0) # array([min_x, min_y, min_z])
bmax   = coords.max(axis=0)
print(bmin, bmax)

In [ ]:
def walls(x):
  #y = 0 or y = 1
  #z = 0, z = 1
  #Says at any X[0] = 0 it is not triggered to make sure no contradictory boundry conditions
  return np.logical_and(
      np.logical_or(
        np.logical_or(np.isclose(x[1], 0, atol=err), np.isclose(x[1], 1, atol=err)),
        np.logical_or(np.isclose(x[2], 0, atol=err), np.isclose(x[2], 1, atol=err))
        ),
      np.logical_and(
          ~np.isclose(x[0], 0, atol=err),
          ~np.isclose(x[0], 1, atol=err)
        )
      )
def inflow(x):
  #x = 0
  return np.isclose(x[0], 0, atol=err)
def outflow(x):
  #x = 1
  return np.isclose(x[0], 1, atol=err)


In [ ]:
wall_dofs = locate_dofs_geometrical(V, walls)
wall_val = Constant(mesh, PETSc.ScalarType((0, 0, 0))) #np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
bc_wall = dirichletbc(wall_val, wall_dofs, V)

inflow_dofs = locate_dofs_geometrical(V, inflow)
inflow_val = Constant(mesh, PETSc.ScalarType((1, 0, 0))) #np.array((1, 0, 0), dtype=PETSc.ScalarType)
bc_inflow = dirichletbc(inflow_val, inflow_dofs, V)

outflow_dofs = locate_dofs_geometrical(V, outflow)
outflow_val = Constant(mesh, PETSc.ScalarType((1, 0, 0))) #np.array((1, 0, 0), dtype = PETSc.ScalarType)
bc_outflow = dirichletbc(outflow_val, outflow_dofs, V)


In [ ]:
bcu = [bc_wall, bc_inflow, bc_outflow]
bcp = []
bcu=[]

In [ ]:
u_n = Function(V)
u_n.name = "u_n"
U = 0.5 * (u_n + u)
n = FacetNormal(mesh)
f = Constant(mesh, PETSc.ScalarType((0, 0, 0)))
k = Constant(mesh, PETSc.ScalarType(dt))
mu = Constant(mesh, PETSc.ScalarType(1))
rho = Constant(mesh, PETSc.ScalarType(1))

In [ ]:
# Define strain-rate tensor
def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor
def sigma(u, p):
    return 2 * mu * epsilon(u) - p * Identity(len(u))


# Define the variational problem for the first step
p_n = Function(Q)
p_n.name = "p_n"
F1 = rho * dot((u - u_n) / k, v) * dx
F1 += rho * dot(dot(u_n, nabla_grad(u_n)), v) * dx
F1 += inner(sigma(U, p_n), epsilon(v)) * dx
F1 += dot(p_n * n, v) * ds - dot(mu * nabla_grad(U) * n, v) * ds
F1 -= dot(f, v) * dx
a1 = form(lhs(F1))
L1 = form(rhs(F1))

In [ ]:
A1 = assemble_matrix(a1, bcs=bcu)
A1.assemble()
b1 = assemble_vector(L1) #-> assemble_vector_pns
print(A1.size)
print(b1.array.shape)

In [ ]:
# Define variational problem for step 2
u_ = Function(V)
a2 = form(dot(nabla_grad(p), nabla_grad(q)) * dx)
L2 = form(dot(nabla_grad(p_n), nabla_grad(q)) * dx - (rho / k) * div(u_) * q * dx)
A2 = assemble_matrix(a2, bcs=bcp)
A2.assemble()
b2 = create_vector(Q, L2)

# Define variational problem for step 3
p_ = Function(Q)
a3 = form(rho * dot(u, v) * dx)
L3 = form(rho * dot(u_, v) * dx - k * dot(nabla_grad(p_ - p_n), v) * dx)
A3 = assemble_matrix(a3, bcs=bcu)
A3.assemble()
b3 = create_vector(V, L3)

In [ ]:
# Solver for step 1
solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(A1)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = PETSc.KSP().create(mesh.comm)
solver2.setOperators(A2)
solver2.setType(PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = PETSc.KSP().create(mesh.comm)
solver3.setOperators(A3)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

In [ ]:
pv.start_xvfb()
visualize = pv.Plotter(off_screen=True)
visualize.open_gif("navier_stokes_cube.gif", fps=10)

topology, cell_types, geometry = vtk_mesh(mesh)

# Create PyVista UnstructuredGrid
pv_mesh = pv.UnstructuredGrid(topology, cell_types, geometry)

actor = visualize.add_mesh(pv_mesh)
actor.visibility = False



In [ ]:
def sinusoidal(x):
  num_points = x.shape[1] # Get the number of points from the shape of x
  return np.vstack((np.sin(x[0] + np.pi),
                    np.cos(x[1] + np.pi),
                    np.full(num_points, 0.))) # Create an array of zeros with the correct number of elements
u0 = Function(V)
u0.interpolate(lambda x: sinusoidal(x))
u0.x.scatter_forward()

In [ ]:
t=0
for i in range(num_steps):
    # Update current time step
    print(f"Flow @ {float(t):.3f}")
    visualize.add_title(f"Flow @ {float(t):.3f}", font_size=16)
    t += dt

    # Step 1: Tentative veolcity step
    with b1.localForm() as loc_1:
        loc_1.set(0)
    assemble_vector(b1, L1) # -> assemble_vector_pns
    apply_lifting(b1, [a1], [bcu]) #in assemble vector pns
    b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE) #in pns Jacobian? seems to be for parallel processing?
    set_bc(b1, bcu) #in assemble vector pns
    solver1.solve(b1, u_.x.petsc_vec)
    print(b1.getSize(), u_.x.petsc_vec.getSize())
    u_.x.scatter_forward()


    # Step 2: Pressure corrrection step
    with b2.localForm() as loc_2:
        loc_2.set(0)
    assemble_vector(b2, L2)
    apply_lifting(b2, [a2], [bcp])
    b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b2, bcp)
    solver2.solve(b2, p_.x.petsc_vec)
    p_.x.scatter_forward()

    # Step 3: Velocity correction step
    with b3.localForm() as loc_3:
        loc_3.set(0)
    assemble_vector(b3, L3)
    b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b3, bcu)
    solver3.solve(b3, u_.x.petsc_vec)
    u_.x.scatter_forward()
    # Update variable with solution form this time step
    u_n.x.array[:] = u_.x.array[:]
    p_n.x.array[:] = p_.x.array[:]


    #Reshapes into vectors
    if i == 0:
      print(u_n.x.array.shape)
    vectors = u_n.x.array.reshape((-1, 3))
    pv_mesh["vectors"] = vectors

    glyphs = pv_mesh.glyph(orient="vectors", scale=False, factor=0.05)
    actor = visualize.add_mesh(glyphs)
    visualize.write_frame()
    visualize.clear()



In [ ]:
b1.destroy()
b2.destroy()
b3.destroy()
solver1.destroy()
solver2.destroy()
solver3.destroy()

In [ ]:
visualize.show(interactive=True)

In [ ]:
visualize.close()

In [ ]:
gif = imageio.mimread("navier_stokes_cube.gif")  # read all frames
imageio.mimsave("slow_gif.gif", gif, duration=0.001) #doesnt work